# E-commerce Sales - Data Exploration

**Objective:** Explore and understand the raw dataset before cleaning and analysis.

**Dataset:** UCI Online Retail II  
**Author:** [Your Name]  
**Date:** [Date]

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 50)

print('✅ Libraries loaded')

In [ ]:
# Load data
# Option 1: Full dataset (download from UCI)
try:
    df = pd.read_excel('../data/raw/online_retail_II.xlsx', sheet_name='Year 2010-2011')
    print(f'✅ Loaded full dataset: {len(df):,} rows')
except FileNotFoundError:
    # Option 2: Sample data
    df = pd.read_csv('../data/sample/sample_data.csv')
    print(f'⚠️ Using sample data: {len(df):,} rows')

df.head()

## 2. Basic Structure

In [ ]:
print('📊 Dataset Structure')
print('=' * 50)
print(f'Rows: {len(df):,}')
print(f'Columns: {len(df.columns)}')
print(f'\nColumn Names:')
for col in df.columns:
    print(f'  - {col}')

In [ ]:
# Data types
print('\n📋 Data Types')
print('=' * 50)
print(df.dtypes)

In [ ]:
# Memory usage
print(f'\n💾 Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')

## 3. Missing Values

In [ ]:
# Missing values summary
missing = pd.DataFrame({
    'Missing': df.isnull().sum(),
    'Percent': (df.isnull().sum() / len(df) * 100).round(2)
}).sort_values('Missing', ascending=False)

print('📋 Missing Values')
print('=' * 50)
print(missing[missing['Missing'] > 0])

In [ ]:
# Visualize missing values
if missing['Missing'].sum() > 0:
    fig, ax = plt.subplots(figsize=(10, 4))
    missing_cols = missing[missing['Missing'] > 0]
    ax.barh(missing_cols.index, missing_cols['Percent'])
    ax.set_xlabel('Missing %')
    ax.set_title('Missing Values by Column', fontweight='bold')
    plt.tight_layout()
    plt.show()

print('\n💡 INSIGHT: Customer ID has missing values — these are likely guest checkouts')

## 4. Numeric Columns

In [ ]:
# Numeric summary
print('📊 Numeric Summary')
print('=' * 50)
df.describe()

In [ ]:
# Check for anomalies
print('\n⚠️ Potential Issues')
print('=' * 50)

# Negative quantities
neg_qty = (df['Quantity'] < 0).sum()
print(f'Negative Quantity: {neg_qty:,} rows ({neg_qty/len(df)*100:.1f}%)')

# Zero or negative prices
zero_price = (df['Price'] <= 0).sum()
print(f'Zero/Negative Price: {zero_price:,} rows ({zero_price/len(df)*100:.1f}%)')

print('\n💡 INSIGHT: Negative quantities are likely returns/cancellations')

In [ ]:
# Distribution plots
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Quantity distribution (clip for visibility)
qty_clipped = df['Quantity'].clip(-100, 500)
axes[0].hist(qty_clipped, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Quantity')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Quantity Distribution (clipped)', fontweight='bold')

# Price distribution (clip for visibility)
price_clipped = df['Price'].clip(0, 50)
axes[1].hist(price_clipped, bins=50, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Price (£)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Price Distribution (clipped at £50)', fontweight='bold')

plt.tight_layout()
plt.savefig('../docs/img/distributions.png', dpi=150)
plt.show()

## 5. Categorical Columns

In [ ]:
# Unique values
print('📊 Categorical Columns')
print('=' * 50)
print(f"Unique Invoices: {df['Invoice'].nunique():,}")
print(f"Unique Stock Codes: {df['StockCode'].nunique():,}")
print(f"Unique Customers: {df['Customer ID'].nunique():,}")
print(f"Unique Countries: {df['Country'].nunique()}")

In [ ]:
# Country distribution
print('\n🌍 Top 10 Countries')
print('=' * 50)
country_counts = df['Country'].value_counts().head(10)
print(country_counts)

In [ ]:
# Invoice patterns
print('\n📋 Invoice Patterns')
print('=' * 50)

# Cancellations (start with 'C')
df['IsCancellation'] = df['Invoice'].astype(str).str.startswith('C')
cancellations = df['IsCancellation'].sum()
print(f"Cancellations: {cancellations:,} rows ({cancellations/len(df)*100:.1f}%)")

print('\n💡 INSIGHT: Cancellations need to be handled separately in analysis')

## 6. Date/Time Analysis

In [ ]:
# Convert to datetime if needed
if df['InvoiceDate'].dtype == 'object':
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print('📅 Date Range')
print('=' * 50)
print(f"Start: {df['InvoiceDate'].min()}")
print(f"End: {df['InvoiceDate'].max()}")
print(f"Span: {(df['InvoiceDate'].max() - df['InvoiceDate'].min()).days} days")

In [ ]:
# Transactions over time
daily_orders = df.groupby(df['InvoiceDate'].dt.date).size()

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(daily_orders.index, daily_orders.values, linewidth=0.8)
ax.set_xlabel('Date')
ax.set_ylabel('Transactions')
ax.set_title('Daily Transaction Volume', fontweight='bold')
plt.tight_layout()
plt.savefig('../docs/img/daily_transactions.png', dpi=150)
plt.show()

## 7. Data Quality Summary

In [ ]:
print('=' * 60)
print('📊 DATA QUALITY SUMMARY')
print('=' * 60)

print(f"""
DATASET OVERVIEW:
• Total rows: {len(df):,}
• Date range: {df['InvoiceDate'].min().date()} to {df['InvoiceDate'].max().date()}
• Countries: {df['Country'].nunique()}
• Customers: {df['Customer ID'].nunique():,}

DATA QUALITY ISSUES:
1. Missing Customer IDs: {df['Customer ID'].isnull().sum():,} rows ({df['Customer ID'].isnull().mean()*100:.1f}%)
   → Likely guest checkouts, exclude from customer analysis

2. Negative Quantities: {neg_qty:,} rows ({neg_qty/len(df)*100:.1f}%)
   → Returns/cancellations, handle separately

3. Zero/Negative Prices: {zero_price:,} rows ({zero_price/len(df)*100:.1f}%)
   → Free items or data entry errors, review before excluding

4. Cancellations: {cancellations:,} rows ({cancellations/len(df)*100:.1f}%)
   → Invoices starting with 'C', exclude from revenue calculations

NEXT STEPS:
→ Proceed to 02_data_cleaning.ipynb to address these issues
""")